In [ ]:
# Following https://github.com/huggingface/notebooks/blob/master/examples/translation.ipynb

In [ ]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
!unzip lc-quad-wikidata-2021-07-09.zip

Archive:  lc-quad-wikidata-2021-07-09.zip
 extracting: lc-quad-wikidata-2021-07-09/dataset_dict.json  
   creating: lc-quad-wikidata-2021-07-09/test/
  inflating: lc-quad-wikidata-2021-07-09/test/dataset.arrow  
  inflating: lc-quad-wikidata-2021-07-09/test/dataset_info.json  
  inflating: lc-quad-wikidata-2021-07-09/test/state.json  
   creating: lc-quad-wikidata-2021-07-09/train/
  inflating: lc-quad-wikidata-2021-07-09/train/dataset.arrow  
  inflating: lc-quad-wikidata-2021-07-09/train/dataset_info.json  
  inflating: lc-quad-wikidata-2021-07-09/train/state.json  


In [ ]:
# !nvidia-smi

In [ ]:
model_checkpoint = 't5-base'
gdir = 'drive/My Drive/Colab Notebooks/t5-2021-07-11/'
model_name='sparql-translator-t5-2021-08-08'
model_path='./models/'+model_name
ds_path= 'lc-quad-wikidata-2021-07-09'

In [ ]:
!mkdir models

mkdir: cannot create directory ‘models’: File exists


In [ ]:
!pip install datasets transformers sacrebleu -qqq

In [ ]:
from datasets import load_dataset, load_metric, Dataset, load_from_disk
raw_datasets = load_from_disk(ds_path)

In [ ]:
!pip install tqdm==4.49.0

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 385, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/comm

In [ ]:
from tqdm import tqdm

In [ ]:
metric = load_metric("sacrebleu")

In [ ]:
# Preprocessing

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
print(raw_datasets['test']['translation'][0]['sparql'])

select distinct ?sbj where [ ?sbj wdt:head_of_state wd:mahmoud_abbas . ?sbj wdt:instance_of wd:country ]


In [ ]:
tokenizer.decode(tokenizer(raw_datasets['test']['translation'][0]['sparql'])['input_ids'])

'select distinct?sbj where [?sbj wdt:head_of_state wd:mahmoud_abbas.?sbj wdt:instance_of wd:country ]</s>'

In [ ]:
print(raw_datasets['test']['translation'][0]['en'],tokenizer(raw_datasets['test']['translation'][0]['en']))

Who is the  {country} for {head of state} of {Mahmoud Abbas} {'input_ids': [2645, 19, 8, 3, 2, 17529, 2, 21, 3, 2, 3313, 13, 538, 2, 13, 3, 2, 329, 9, 107, 51, 1063, 26, 891, 4883, 2, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
prefix = "translate English to Sparql: "

In [ ]:
max_input_length = 0 
max_target_length = 0
for d in tqdm(raw_datasets['train']['translation']):
    len_en = len(d['en'])
    len_qry = len(d['sparql'])
    if len_en > max_input_length: max_input_length=len_en
    if len_qry > max_target_length: max_target_length=len_qry

100%|██████████| 38175/38175 [00:00<00:00, 1208990.97it/s]


In [ ]:
print(max_input_length, max_target_length)

248 349


In [ ]:
source_lang = "en"
target_lang = "sparql"

def preprocess_function(examples):
    inputs = []
    targets= []
    for ex in examples["translation"]:
      inputs.append(prefix + ex[source_lang])
      targets.append(ex[target_lang])
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
      labels = tokenizer(targets, max_length=max_target_length, truncation=True)
      
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'translation'],
        num_rows: 38175
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'translation'],
        num_rows: 9485
    })
})

In [ ]:
# Fine-tuning the model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
batch_size = 8
args = Seq2SeqTrainingArguments(
    model_name,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = [["hello there"], ["general kenobi"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'bp': 1.0,
 'counts': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'ref_len': 4,
 'score': 0.0,
 'sys_len': 4,
 'totals': [4, 2, 0, 0]}

In [ ]:
# The last thing to define for our Seq2SeqTrainer is how to compute 
# the metrics from the predictions. We need to define 
# a function for this, which will just use the metric we loaded earlier, 
# and we have to do a bit of pre-processing to decode the predictions into texts:

import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.replace('.',' ').replace('?',' ').strip() for pred in preds]
    labels = [[label.replace('.',' ').replace('?',' ').strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    # decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)
    # compute_metrics=compute_metrics

Using amp fp16 backend


In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: translation.
***** Running training *****
  Num examples = 38175
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 4772


Epoch,Training Loss,Validation Loss
1,0.208900,0.180047


Saving model checkpoint to sparql-translator-t5-2021-08-08/checkpoint-500
Configuration saved in sparql-translator-t5-2021-08-08/checkpoint-500/config.json
Model weights saved in sparql-translator-t5-2021-08-08/checkpoint-500/pytorch_model.bin
tokenizer config file saved in sparql-translator-t5-2021-08-08/checkpoint-500/tokenizer_config.json
Special tokens file saved in sparql-translator-t5-2021-08-08/checkpoint-500/special_tokens_map.json
Deleting older checkpoint [sparql-translator-t5-2021-08-08/checkpoint-3500] due to args.save_total_limit
Saving model checkpoint to sparql-translator-t5-2021-08-08/checkpoint-1000
Configuration saved in sparql-translator-t5-2021-08-08/checkpoint-1000/config.json
Model weights saved in sparql-translator-t5-2021-08-08/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in sparql-translator-t5-2021-08-08/checkpoint-1000/tokenizer_config.json
Special tokens file saved in sparql-translator-t5-2021-08-08/checkpoint-1000/special_tokens_map.json
De

TrainOutput(global_step=4772, training_loss=0.19678540481605752, metrics={'train_runtime': 1486.2367, 'train_samples_per_second': 25.686, 'train_steps_per_second': 3.211, 'total_flos': 6833255646366720.0, 'train_loss': 0.19678540481605752, 'epoch': 1.0})

In [ ]:
trainer.save_model(model_path)

In [ ]:
!ls -l --block-size=M {model_path}

In [ ]:
!cp {model_path}/* drive/MyDrive/models/